In [2]:
import cv2
import numpy as np
from math import sqrt
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

class detobj:
    def __init__(self, nID, nX, nY):
        self.id = nID
        self.x = nX
        self.y = nY
        self.remove = 0
        self.start = 0

    def Show(self):
        print(f'id={self.id} x={self.x} y={self.y}')

    def update(self, x, y):
        self.x = x
        self.y = y
        self.remove = 0

    def clear(self):
        self.remove = 1

    def Cross(self, Line):
        if self.y < Line:
            self.start = 0
        else:
            self.start = 1
            
source = 'C:/Users/nikhi/Documents/S6/Computer Vision/Projects/highway.mp4'
cap = cv2.VideoCapture(source)
imgsz = 640
conf_thres = 0.25
iou_thres = 0.45
trobjs = []
aid = 0
LineStart = (300, 400)
LineStop = (1000, 400)
HudFont = cv2.FONT_HERSHEY_SIMPLEX
HudFontScale = 1
HudDown = (280, 200)
HudUp = (1020, 200)
HudColor = (0, 255, 0)
HudThickness = 2
CountUp = 0
CountDown = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)
    new_trobjs = []
    for result in results:
        for det in result.boxes.data.cpu().numpy():
            x1, y1, x2, y2, conf, cls = det[:6]
            if conf < conf_thres:
                continue
            xc = (x1 + x2) / 2
            yc = (y1 + y2) / 2
            curobj = None
            min_dist = float('inf')
            for ob in trobjs:
                d = sqrt((xc - ob.x) ** 2 + (yc - ob.y) ** 2)
                if d < min_dist and d < 40:
                    min_dist = d
                    curobj = ob
            if curobj is not None:
                curobj.update(xc, yc)
            else:
                curobj = detobj(aid, xc, yc)
                curobj.Cross(LineStart[1])
                aid += 1
            new_trobjs.append(curobj)
            label = f'id{curobj.id} {model.names[int(cls)]}'
            frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color=(255, 0, 0), thickness=2)
            frame = cv2.putText(frame, label, (int(x1), int(y1) - 10), HudFont, HudFontScale, (255, 0, 0), HudThickness)
    trobjs = new_trobjs
    
    for ob in trobjs:
        if ob.start == 0 and ob.y > LineStart[1]:
            CountDown += 1
            ob.start = 1
        elif ob.start == 1 and ob.y <= LineStart[1]:
            CountUp += 1
            ob.start = 0

    frame = cv2.line(frame, LineStart, LineStop, (0, 255, 0), thickness=3)
    frame = cv2.putText(frame, str(CountDown), HudDown, HudFont, HudFontScale, HudColor, HudThickness, cv2.LINE_AA)
    frame = cv2.putText(frame, str(CountUp), HudUp, HudFont, HudFontScale, HudColor, HudThickness, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('YOLOv8 Detection', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 4 cars, 133.0ms
Speed: 3.0ms preprocess, 133.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 180.6ms
Speed: 5.0ms preprocess, 180.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 148.0ms
Speed: 4.0ms preprocess, 148.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 154.7ms
Speed: 2.0ms preprocess, 154.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 150.9ms
Speed: 2.0ms preprocess, 150.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 138.6ms
Speed: 2.0ms preprocess, 138.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 144.0ms
Speed: 2.0ms preprocess, 144.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 154.2ms
Speed: 3.0ms preprocess, 154.2ms inference, 1.0ms postprocess per image at shape (1, 3, 38